# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [2]:
# imports
import os
from uuid import uuid4
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from openai import OpenAIError, APIConnectionError
from uuid import uuid4
from IPython.display import display, Markdown, update_display

In [3]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [4]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI()

API key looks good so far


In [5]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}



In [6]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
suc

In [7]:
messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [8]:

ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL_LLAMA,
    messages=messages
)

print(response.choices[0].message.content)

Generative AI has numerous business applications across various industries, including:

1. **Content Generation**: Create high-quality content such as articles, social media posts, product descriptions, and more. For example, AI-powered tools like WordLift can generate well-researched and optimized blog posts in minutes.
2. **Social Media Management**: Use generative AI to create engaging social media content, respond to customer inquiries, and monitor social media sentiment analysis.
3. **Marketing Automation**: Automate marketing tasks such as lead generation, email marketing, and ad copywriting using generative AI-powered tools like HubSpot's Content Generator.
4. **Product Design**: Collaborate with designers and engineers to create product designs using generative AI tools like Autodesk's Dreambook. These tools can generate 3D models and design suggestions based on customer preferences and market trends.
5. **Data Analysis and Visualization**: Use generative AI-powered data analys

In [24]:
# here is the question; type over this to ask something new

question = """
I'm sttuck with how to connect ollama locally and use in my python code 
import requests
import json
from uuid import uuid4
from IPython.display import display, Markdown

# Constants
MODEL_LLAMA = 'llama3.2'  # Ensure this is the correct model name
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}

# Prepare user message
user_message = user_prompt.format(question=question.strip())
display_id = str(uuid4())
output_buffer = ""

# Display initial placeholder
display(Markdown("**Response:**"), display_id=display_id)

try:
    # Construct the payload for the request
    payload = {
        "model": MODEL_LLAMA,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    }

    # Call the API with streaming
    response = requests.post(OLLAMA_API, headers=HEADERS, data=json.dumps(payload), stream=True)

    # Check for successful response
    response.raise_for_status()  # Raises an error for 4XX/5XX responses

    # Process and stream output
    for chunk in response.iter_lines():
        if chunk:
            data = json.loads(chunk)
            if 'choices' in data and len(data['choices']) > 0:
                content = data['choices'][0].get('delta', {}).get('content', '')
                if content:
                    output_buffer += content
                    update_display(Markdown("**Response:** " + output_buffer), display_id=display_id)

except requests.exceptions.HTTPError as e:
    update_display(Markdown(f"**Error Code:** {e.response.status_code} - {e.response.text}"), display_id=display_id)
except Exception as e:
    update_display(Markdown(f"**Unexpected Error:** {str(e)}"), display_id=display_id)

The code does not display anything even errors just empty response

maybe remove the streaming feature and just return the response 


"""

In [25]:
system_prompt = """You are a highly skilled coding assistanti. Your job is to help the user write, debug, and understand code. Provide accurate, concise, and
production-grade answers in real time. Prioritize clarity, best practices, and relevance. Use comments to explain complex logic where needed. If the user gives
incomplete information, ask clarifying questions before answering. You can write and explain code in Python, JavaScript, C++, Bash, and other major languages as requested.

Always format code in markdown blocks. Be honest when you don't know something or when the user input is ambiguous. Your responses should stream continuously and naturally.
"""

user_prompt = """I'm coding and need help. Please answer the following clearly and correctly:

{question} 
"""

In [26]:


# Set up
user_message = user_prompt.format(question=question.strip())
display_id = str(uuid4())
output_buffer = ""

# Display initial placeholder
display(Markdown("**Response:**\n"), display_id=display_id)

try:
    # Call the API with streaming
    response = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ],
        stream=True
    )

    # Process and stream output
    for chunk in response:
        if chunk.choices[0].delta.content:
            output_buffer += chunk.choices[0].delta.content
            update_display(Markdown("**Response:**\n" + output_buffer), display_id=display_id)

except APIConnectionError:
    update_display(Markdown("**Error:** Network is unreachable. Please check your internet connection."), display_id=display_id)

except OpenAIError as e:
    update_display(Markdown(f"**Error:** {str(e)}"), display_id=display_id)

except Exception as e:
    update_display(Markdown(f"**Unexpected Error:** {str(e)}"), display_id=display_id)


**Response:**
It looks like you're attempting to connect to a local Ollama API using Python and you're facing issues displaying the response. If you want to modify your code to handle the API request without streaming, you can simplify it. Let's make those changes.

Here's a version of your code that sends the request and returns the complete response without streaming:

```python
import requests
import json
from uuid import uuid4
from IPython.display import display, Markdown

# Constants
MODEL_LLAMA = 'llama3.2'  # Ensure this is the correct model name
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}

# Prepare user message
user_message = user_prompt.format(question=question.strip())
display_id = str(uuid4())

# Display initial placeholder
display(Markdown("**Response:**"), display_id=display_id)

try:
    # Construct the payload for the request
    payload = {
        "model": MODEL_LLAMA,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    }

    # Call the API
    response = requests.post(OLLAMA_API, headers=HEADERS, data=json.dumps(payload))

    # Check for successful response
    response.raise_for_status()  # Raises an error for 4XX/5XX responses

    # Process the response
    data = response.json()  # Parse the JSON response directly
    if 'choices' in data and len(data['choices']) > 0:
        content = data['choices'][0].get('message', {}).get('content', '')  # Adjust based on response structure
        if content:
            display(Markdown("**Response:** " + content), display_id=display_id)

except requests.exceptions.HTTPError as e:
    display(Markdown(f"**Error Code:** {e.response.status_code} - {e.response.text}"), display_id=display_id)
except Exception as e:
    display(Markdown(f"**Unexpected Error:** {str(e)}"), display_id=display_id)
```

### Changes Made:
1. **Removed Streaming**: The response is now fetched in a single request without streaming. The API call is made using a standard `requests.post()` call and then we process the entire response after waiting for it to complete.
   
2. **Response Handling**: After checking for a successful response, we directly access the JSON data. Ensure that you adapt the response access (`data['choices'][0].get('message', {}).get('content', '')`) according to the actual structure of the response you receive. If the structure differs, adjust this line accordingly.

### Troubleshooting
1. **Check API Status**: Ensure your Ollama API is running and reachable at `http://localhost:11434/api/chat`.

2. **Verify Model Name**: Make sure that the model name `llama3.2` is correct, as a wrong model name can also lead to unexpected issues.

3. **Test API Endpoint**: You can manually test the API using a tool like Postman or curl to see if it returns the expected response.

If you still face issues, please provide any specific error messages or responses you receive for further assistance!

In [30]:
#Using Ollama

user_message = user_prompt.format(question=question.strip())
display_id = str(uuid4())
output_buffer = ""

display(Markdown("**Response:**\n"), display_id=display_id)

try:
    
    payload = {
        "model": MODEL_LLAMA,
        "stream": True,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    }

    response = requests.post(OLLAMA_API, headers=HEADERS, data=json.dumps(payload), stream=True)
    response.raise_for_status()

    for line in response.iter_lines():
        if line:
            try:
                chunk = json.loads(line.decode("utf-8"))
                if "message" in chunk and "content" in chunk["message"]:
                    output_buffer += chunk["message"]["content"]
                    update_display(Markdown("**Response:**\n" + output_buffer), display_id=display_id)
            except json.JSONDecodeError:
                continue

except requests.exceptions.ConnectionError:
    update_display(Markdown("**Error:** Network is unreachable. Make sure Ollama is running."), display_id=display_id)

except requests.exceptions.HTTPError as e:
    update_display(Markdown(f"**HTTP Error {e.response.status_code}:** {e.response.text}"), display_id=display_id)

except Exception as e:
    update_display(Markdown(f"**Unexpected Error:** {str(e)}"), display_id=display_id)


**Response:**
It seems like you're trying to use the LLaMA API in your Python application. To simplify things, let's try removing the `stream=True` parameter from the `requests.post()` call. This will allow us to get the full response instead of processing it as it's coming in.

Here are some changes I'd suggest making:

```python
# Construct the payload for the request
payload = {
    "model": MODEL_LLAMA,
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]
}

# Call the API
response = requests.post(OLLAMA_API, headers=HEADERS, data=json.dumps(payload))

# Check for successful response
if response.status_code == 200:
    # Process and display output
    data = json.loads(response.text)
    if 'choices' in data and len(data['choices']) > 0:
        content = data['choices'][0].get('delta', {}).get('content', '')
        if content:
            output_buffer += content
            update_display(Markdown("**Response:** " + output_buffer), display_id=display_id)
else:
    # Display error message
    update_display(Markdown(f"**Error Code:** {response.status_code} - {response.text}"), display_id=display_id)
```

This should help you see if the issue is with the API call or something else. Also, note that I removed the `raise_for_status()` line because we're checking the status code manually.

Additionally, consider adding a try-except block around `json.loads(response.text)` to handle any potential errors that might occur when parsing the JSON response.